<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 1 / NOTE 3*

# 📝 Assignment
---

# Ridge Regression

이번 모듈에서 배운 내용을 조금 다른 데이터를 통해 복습해 봅시다. 데이터에 대한 정보는 다음 링크를 참조해 주세요.

[Melbourne Housing Market]('https://www.kaggle.com/anthonypino/melbourne-housing-market')

이번에도 Price 변수를 target으로 정하고 모델을 구성해 봅시다.

In [4]:
# RidgeCV better_score_ 오류 해결 -> version up! 
!pip install -U scikit-learn # !pip install --upgrade scikit-learn==0.24.1

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [5]:
# 필요한 라이브러리 import 
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import pandas as pd

In [20]:
# get data
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/melbourne_house_prices/MELBOURNE_HOUSE_PRICES_LESS.csv')

# information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63023 entries, 0 to 63022
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         63023 non-null  object 
 1   Address        63023 non-null  object 
 2   Rooms          63023 non-null  int64  
 3   Type           63023 non-null  object 
 4   Price          48433 non-null  float64
 5   Method         63023 non-null  object 
 6   SellerG        63023 non-null  object 
 7   Date           63023 non-null  object 
 8   Postcode       63023 non-null  int64  
 9   Regionname     63023 non-null  object 
 10  Propertycount  63023 non-null  int64  
 11  Distance       63023 non-null  float64
 12  CouncilArea    63023 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 6.3+ MB


In [21]:
# drop high cardinality cols
df.drop(columns=['Suburb','Address','SellerG','Date'], inplace=True)

# drop na values in Price
df.dropna(subset=['Price'], inplace=True)

# drop duplicated rows
df.drop_duplicates(inplace=True)

In [8]:
print(df.shape)
df.head()

(42515, 9)


,Rooms,Type,Price,Method,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,3,h,1490000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,3,h,1220000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,3,h,1420000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,3,h,1515000.0,S,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,2,h,670000.0,S,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council


### 1) 데이터의 특성을 파악하기 위해 pandas_profiling을 사용해 보세요.
- **범주형 특성으로 원핫인코딩이 꼭 필요한 특성의 갯수를 과제 제출폼에 제출하세요.**

# feature 
>Type
- br: bedroom(s) 
- h : house,cottage,villa, semi,terrace
- u : unit, duplex
- t : townhouse
- dev site : development site
- o res : other residential

>Rooms
: Number of rooms

>Method
- S : property sold
- SP : property sold prior
- PI : property passed in
- PN : sold prior not disclosed
- SN : sold not disclosed
- NB : no bid
- VB : vendor bid
- W : withdrawn prior to auction
- SA : sold after auction
- SS : sold after auction price not disclosed
- N/A : price or highest bid not available

>Regionname
: General Region (West, North West, North, North east …etc)

>Propertycount
: Number of properties that exist in the suburb.

>Distance
: Distance from CBD in Kilometres

>CouncilArea
: Governing council for the area

In [ ]:
# install pandas profiling library
!pip install -U pandas-profiling

In [ ]:
import pandas_profiling
# from pandas_profiling import ProfileReport

pr=df.profile_report() # ProfileReport(df)
pr

# 범주형 변수로 꼭 one hot encoding 필요한 특성의 수 : 4 

### 2) 범주형 변수들에 대해 one-hot encoding(category_encoders)을 수행하고 데이터를 train/test 셋으로 무작위로 나누어 주세요.(train/test = 80%/20%)
- [category_encoders.one_hot.OneHotEncoder](https://contrib.scikit-learn.org/category_encoders/onehot.html#one-hot), 파라미터는 use_cat_names = True 사용
- 데이터 분리는 [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train%20test%20split#sklearn-model-selection-train-test-split) 을 사용해 주세요. 사용해야 하는 파라미터는 test_size, train_size, random_state=2 입니다. 나머지는 default 사용합니다.
- **분리된 학습데이터의 첫번째 샘플의 인덱스 값를 과제 제출폼에 제출하세요.**


In [12]:
# install category_encoders library
!pip install category_encoders

     |████████████████████████████████| 81kB 3.6MB/s 


In [22]:
df.columns

Index(['Rooms', 'Type', 'Price', 'Method', 'Postcode', 'Regionname',
       'Propertycount', 'Distance', 'CouncilArea'],
      dtype='object')

In [25]:
## import OneHotEncoder
from category_encoders import OneHotEncoder
from sklearn.model_selection import train_test_split

# target & features
target = ['Price']
features = ['Rooms', 'Type', 'Method', 'Postcode', 'Regionname',
       'Propertycount', 'Distance', 'CouncilArea']

df_features = df[features]
df_target = df[target]

# train, test split
X_train, X_test, y_train, y_test = train_test_split(df_features,df_target,test_size = 0.2,random_state = 2)

## 원핫 인코딩
encoder = OneHotEncoder(use_cat_names = True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

print(X_train.shape)
X_train.head() # 분리된 학습데이터의 첫번째 샘플의 인덱스 : 602

# Future Warning: 나중에 version이 다시 업데이트되면 안될 수 있으니깐 조심해~ 

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


(34012, 54)


,Rooms,Type_h,Type_u,Type_t,Method_S,Method_VB,Method_SP,Method_PI,Method_SA,Postcode,Regionname_Eastern Metropolitan,Regionname_South-Eastern Metropolitan,Regionname_Western Metropolitan,Regionname_Northern Metropolitan,Regionname_Southern Metropolitan,Regionname_Eastern Victoria,Regionname_Western Victoria,Regionname_Northern Victoria,Propertycount,Distance,CouncilArea_Manningham City Council,CouncilArea_Kingston City Council,CouncilArea_Hobsons Bay City Council,CouncilArea_Brimbank City Council,CouncilArea_Whittlesea City Council,CouncilArea_Hume City Council,CouncilArea_Stonnington City Council,CouncilArea_Frankston City Council,CouncilArea_Monash City Council,CouncilArea_Darebin City Council,CouncilArea_Boroondara City Council,CouncilArea_Yarra City Council,CouncilArea_Banyule City Council,CouncilArea_Moreland City Council,CouncilArea_Wyndham City Council,CouncilArea_Port Phillip City Council,CouncilArea_Whitehorse City Council,CouncilArea_Glen Eira City Council,CouncilArea_Bayside City Council,CouncilArea_Moonee Valley City Council,CouncilArea_Melbourne City Council,CouncilArea_Maribyrnong City Council,CouncilArea_Knox City Council,CouncilArea_Maroondah City Council,CouncilArea_Yarra Ranges Shire Council,CouncilArea_Melton City Council,CouncilArea_Greater Dandenong City Council,CouncilArea_Casey City Council,CouncilArea_Macedon Ranges Shire Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Cardinia Shire Council,CouncilArea_Moorabool Shire Council,CouncilArea_Mitchell Shire Council,CouncilArea_Murrindindi Shire Council
602,4,1,0,0,1,0,0,0,0,3106,1,0,0,0,0,0,0,0,6202,15.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40851,5,1,0,0,1,0,0,0,0,3169,0,1,0,0,0,0,0,0,2727,17.5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
57233,4,1,0,0,0,1,0,0,0,3028,0,0,1,0,0,0,0,0,7630,15.5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8816,4,1,0,0,1,0,0,0,0,3033,0,0,1,0,0,0,0,0,5629,11.7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3481,3,1,0,0,1,0,0,0,0,3076,0,0,0,1,0,0,0,0,10926,19.6,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 3) SelectKBest를 사용해서 20개의 특성만 selection 하세요
- score_func 에는 f_regression을 사용합니다
- **가장 스코어(Attributes: scores_ 사용)가 높게 나온 특성의 이름을 과제 제출폼에 제출하세요.**

In [27]:
## f_regresison, SelectKBest
from sklearn.feature_selection import f_regression, SelectKBest

## selctor 정의합니다. k = 20
selector = SelectKBest(score_func=f_regression, k=20)

## 학습데이터에 fit_transform 
X_train_selected = selector.fit_transform(X_train, y_train)

## 테스트 데이터는 transform
X_test_selected = selector.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [28]:
all_columns = X_train.columns

# 선택된 컬럼 
selected_columns = all_columns[selector.get_support()]

# 선택되지 않은 컬럼
unselected_columns = all_columns[~(selector.get_support())]

print(f'\n선택된 컬럼 :\n {selected_columns}\n')
print(f'\n선택되지 않은 컬럼 :\n {unselected_columns}\n')

print(f'\nscore가 가장 높게 나온 특성: {all_columns[selector.scores_.argmax()]}\n')


선택된 컬럼 :
 Index(['Rooms', 'Type_h', 'Type_u', 'Method_VB', 'Method_SP',
       'Regionname_South-Eastern Metropolitan',
       'Regionname_Western Metropolitan', 'Regionname_Northern Metropolitan',
       'Regionname_Southern Metropolitan', 'Distance',
       'CouncilArea_Manningham City Council',
       'CouncilArea_Brimbank City Council',
       'CouncilArea_Whittlesea City Council', 'CouncilArea_Hume City Council',
       'CouncilArea_Stonnington City Council',
       'CouncilArea_Frankston City Council',
       'CouncilArea_Boroondara City Council',
       'CouncilArea_Wyndham City Council', 'CouncilArea_Bayside City Council',
       'CouncilArea_Melton City Council'],
      dtype='object')


선택되지 않은 컬럼 :
 Index(['Type_t', 'Method_S', 'Method_PI', 'Method_SA', 'Postcode',
       'Regionname_Eastern Metropolitan', 'Regionname_Eastern Victoria',
       'Regionname_Western Victoria', 'Regionname_Northern Victoria',
       'Propertycount', 'CouncilArea_Kingston City Council',
       '

### 4) RidgeCV를 사용해서 회귀모델을 만들어 보세요
- attribute는 normalize=True, cv=5를 사용하세요. 나머지는 default
- **alphas = [0, 0.001, 0.01, 0.1, 1] 을 시도해보고 가장 좋은 점수가 나온 alpha 값을 과제 제출폼에 제출하세요.**

In [30]:
## score가 가장 높게 나온 feature Rooms와 target data price 사용
from sklearn.linear_model import RidgeCV

alphas = [0, 0.001, 0.01, 0.1, 1]

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5)
# ridge.fit(X_train[['Rooms']], y_train)
ridge.fit(X_train_selected, y_train)
print("alpha: ", ridge.alpha_)
print("best score :", ridge.best_score_)

alpha:  0.001
best score : 0.5510507608242425


### 5) 테스트셋에 대한 MAE 값을 계산하고 모델이 잘 만들어졌는지 확인해 보세요.
- **테스트셋에 대한 MAE 값을 과제 제출폼에 제출하세요.**

In [33]:
## train data MAE
y_tpred = ridge.predict(X_train_selected)
r2 = r2_score(y_train, y_tpred)
mae = mean_absolute_error(y_train, y_tpred)
# print(f'Train MAE: ${mae:,.0f}')
# print(f'Train R2 Score: {r2:,.4f}\n')

## test data MAE
y_epred = ridge.predict(X_test_selected)
r2 = r2_score(y_test, y_epred)
mae = mean_absolute_error(y_test, y_epred)
print(f'Test MAE: {mae}')
# print(f'Test R2 Score: {r2:,.4f}\n')

Test MAE: 262857.3198109039


## 🔥 도전과제

### 6) best ridge regression model을 만드세요.
    - 특성의 갯수에 제한이 없습니다. -> selectKtest로 몇 개의 특성을 선택하면 좋을지 알아보자 .
    - 다항회귀를 사용하세요
    - 여러 람다값을 시도하세요
    - 람다 0인 경우를 비교하세요
    - RidgeCV를 사용하세요
    - 만든 모델의 특징을 간략히 설명하고 테스트세트 성능 결과를 메신저에서 공유하세요

In [ ]:
# features를 몇 개 선책하는 것이 좋은지 알아 봅시다.

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

training = []
testing = []
ks = range(1, len(X_train.columns)+1)

# 다항회귀 & RidgeCV 사용
def RidgeCVRegression(degree=3, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), 
                         RidgeCV(**kwargs))

# 1 부터 특성 수 만큼 사용한 모델을 만들어서 MAE 값을 비교 합니다.
for k in range(1, len(X_train.columns)+ 1):
    print(f'[{k} features]')

    # alphas = np.linspace(0, 0.5, num=20)
    alphas = [0, 0.001, 0.01, 0.1, 1]
    selector = SelectKBest(score_func=f_regression, k=k)
    
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    all_names = X_train.columns
    selected_mask = selector.get_support()
    selected_names = all_names[selected_mask]
    print(f'Selected names: {selected_names.values}')

    model = RidgeCVRegression(alphas=alphas, normalize=True, cv=5)

    # 모델 학습
    model.fit(X_train_selected, y_train)

    # 예측에 train data 사용
    y_pred = model.predict(X_train_selected)

    # MAE & R2 확인
    mae = mean_absolute_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    print(f'Train MAE: ${mae:,.0f}')
    print(f'Train R2: {r2:,.4f}')

    # training.append(mae) # train data mae k = 1 ~ 전체 컬럼의 수까지 리스트에 담기 
    
    # 예측 - test data 사용
    y_pred = model.predict(X_test_selected)

    # MAE & R2 확인
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    testing.append(mae)
    print(f'Test MAE: ${mae:,.0f}')
    print(f'Test R2: {r2:,.4f}')

    coefs = model.named_steps["ridgecv"].coef_
    print(f'Number of Features: {len(coefs)}')
    print(f'alpha: {model.named_steps["ridgecv"].alpha_}')
    print(f'cv best score: {model.named_steps["ridgecv"].best_score_}\n') # best score: R2

    # 만약 feature가 증가했는데 mae값이 더 커지면 break / 더 진행해볼 이유 없다고 판단.
    if len(testing) > 1:
      if testing[k-2] < testing[k-1]:
        print(f'{k}의 feauture까지는 필요 없다.')
        break

[1 features]
Selected names: ['Rooms']
Train MAE: $373,526
Train R2: 0.1753
Test MAE: $380,041
Test R2: 0.1765
Number of Features: 4
alpha: 0.01
cv best score: 0.16161360410703735

[2 features]
Selected names: ['Rooms' 'Regionname_Southern Metropolitan']
Train MAE: $318,896
Train R2: 0.4025
Test MAE: $320,445
Test R2: 0.4191
Number of Features: 10
alpha: 0.1
cv best score: 0.3997809052606929

[3 features]
Selected names: ['Rooms' 'Type_u' 'Regionname_Southern Metropolitan']
Train MAE: $303,009
Train R2: 0.4342
Test MAE: $307,176
Test R2: 0.4454
Number of Features: 20
alpha: 0.01
cv best score: 0.41756347395911425

[4 features]
Selected names: ['Rooms' 'Type_h' 'Type_u' 'Regionname_Southern Metropolitan']
Train MAE: $296,894
Train R2: 0.4539
Test MAE: $300,939
Test R2: 0.4665
Number of Features: 35
alpha: 0.01
cv best score: 0.44247017864409

[5 features]
Selected names: ['Rooms' 'Type_h' 'Type_u' 'Regionname_Southern Metropolitan'
 'CouncilArea_Boroondara City Council']
Train MAE: $292

In [ ]:
X_total = pd.concat([X_train, X_test])
y_total = pd.concat([y_train, y_test])

In [ ]:
# def RidgeCVRegression(degree=3, **kwargs):
#     return make_pipeline(PolynomialFeatures(degree), 
#                          RidgeCV(**kwargs))

# # 모든 데이터를 사용해 최종 모델을 만듭니다.
# model = RidgeCVRegression(alphas=alphas, normalize=True, cv=5)
# model.fit(X_total, y_total)

# coefs = model.named_steps["ridgecv"].coef_
# print(f'Number of Features: {len(coefs)}')

# print(f'alpha: {model.named_steps["ridgecv"].alpha_}')
# # print(f'cv best score: {model.named_steps["ridgecv"].best_score_}')

In [ ]:
# coefs.max(), coefs.mean()